In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
from sys import exc_info

import sqlite3

In [2]:
from Modules.dataEngineering import dataEngineering
dataEngineering = dataEngineering()

In [3]:
movies = dataEngineering.loadMovies()['data']
users = dataEngineering.loadUsers()['data']
ratings = dataEngineering.loadRatings()['data']

In [4]:
if type(movies['Genres'][0]) is not list:
    movies['Genres'] = movies['Genres'].str.split('|')
    print("Change complete")

Change complete


In [5]:
genre_dict = {
    'Adventure': 0,
    'Animation': 1,
    'Children': 2,
    'Comedy': 3,
    'Fantasy': 4,
    'Romance': 5,
    'Drama': 6,
    'Action': 7,
    'Crime': 8,
    'Thriller': 9,
    'Horror': 10,
    'Mystery': 11,
    'Sci-Fi': 12,
    'War': 13,
    'Musical': 14,
    'Documentary': 15,
    'IMAX': 16,
    'Western': 17,
    'Film-Noir': 18,
    '(no genres listed)': 19
}

In [10]:
genres_Movies = movies[["MovieID", "Genres"]].reset_index()

for index, genres_list in genres_Movies[["index", "Genres"]].values:
    vector = list()
    for g in genre_dict:
        if g in genres_list:
            vector.append(1)
        else:
            vector.append(0)
        genres_Movies.at[index, "Genres"] = np.array(vector)

genres_Movies.drop(columns=['index'], inplace=True)
genres_Movies.rename(columns={'Genres':'Movie_Vector'}, inplace=True)

genres_Movies.head(2)

,MovieID,Movie_Vector
0,1,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
vector_movies = pd.merge(movies, genres_Movies, on='MovieID').sort_values(['MovieID'], ascending = [True]).reset_index()
vector_movies.drop(columns=['index', 'Genres', 'Years'], inplace=True)
# vector_movies.drop(columns=['index'], inplace=True)
vector_movies.head(5)

,MovieID,Title,Movie_Vector
0,1,Toy Story,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,Jumanji,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,Grumpier Old Men,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,Waiting to Exhale,"[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,Father of the Bride Part II,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
vector_movies[vector_movies['MovieID'] == 2]['Movie_Vector'][1]

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
vector_ratings = pd.merge(ratings, genres_Movies, on='MovieID').sort_values(['UserID', 'MovieID'], ascending = [True, True]).reset_index()
vector_ratings.drop(columns=['index'], inplace=True)
vector_ratings.head(5)

,UserID,MovieID,userRating,Years,Movie_Vector
0,3,1,4.0,2015,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,3,29,4.5,2017,"[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, ..."
2,3,32,4.5,2015,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, ..."
3,3,50,5.0,2015,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, ..."
4,3,111,4.0,2017,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ..."


In [15]:
vector_ratings['Movie_Vector'][0]

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [40]:
import numpy as np

def user_vector():
    user_ids = vector_ratings['UserID'].sort_values(ascending=True).unique()[:50]

    df_users_3 = pd.DataFrame(columns=['UserID', "User_Vector", "Count_Movies"])

    for uid in user_ids:
        user_ratings = vector_ratings[vector_ratings['UserID'] == uid]

        user_vector = np.zeros(len(genre_dict))
        count_vector = np.zeros(len(genre_dict))

        count_Movies = 0

        for _, row in user_ratings.iterrows():
            count_Movies += 1
            count_vector += row.Movie_Vector
            user_vector += row.Movie_Vector * row.userRating

        count_vector = np.where(count_vector==0, 1, count_vector)
        user_vector = np.divide(user_vector, count_vector)

        user_vector /= 5

        user_vector = np.round(user_vector, 4)

        row_df = pd.DataFrame([[uid, user_vector, count_Movies]], 
                            columns=['UserID', 'User_Vector', "Count_Movies"])

        df_users_3 = pd.concat([df_users_3, row_df], ignore_index=True)

    return df_users_3


In [42]:
df_users_3 = user_vector()
df_users_3.head(5)

,UserID,User_Vector,Count_Movies
0,3,"[0.7402, 0.8045, 0.7545, 0.6906, 0.7472, 0.706...",634
1,4,"[0.6149, 0.6967, 0.6444, 0.7263, 0.5703, 0.69,...",233
2,33,"[0.7, 0.6, 0.6, 0.8, 0.7, 0.8, 0.8412, 0.8, 0....",23
3,63,"[0.95, 0.94, 0.8444, 0.7967, 0.76, 0.7529, 0.7...",40
4,76,"[0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.8, 0.8, ...",2


หาความคล้ายคลี่ง ระหว่าง User กับ Movie โดยใช้ Genres เป็นตัวเปรียบเทียบ

In [43]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
def cosine(df_users):
    M = vector_movies['Movie_Vector'].to_numpy()
    U = df_users['User_Vector'].to_numpy()

    M = np.vstack(M)
    U = np.vstack(U)

    # คำนวณ cosine similarity ระหว่างหนังและผู้ใช้
    similarity_matrix = cosine_similarity(M, U)
    return similarity_matrix

similarity_matrix = cosine(df_users_3)

Save Model

In [45]:
import joblib

joblib.dump(similarity_matrix, 'models/model_Content-Base.pkl')

['models/model_Content-Base.pkl']

In [46]:
model = joblib.load('models/model_Content-Base.pkl')

In [68]:
def get_recommendations(userID, df_users, cosine_sim=model):

    try:
        user_Index = df_users[df_users["UserID"] == userID].UserID.values[0]
    except(KeyError):
        return ("You don't have movie viewing history.")

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[:, user_Index]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[:10]

    # Return the top 10 most similar movies
    index_movies = vector_movies.copy().reset_index()

    top_movies_4u = []
    # top_movies_4u = [index_movies[index_movies["index"] == index].Title.values[0] for index in movie_indices]
    for m_index, score in sim_scores:
        m_id = index_movies[index_movies["index"] == m_index].Title.values[0]
        top_movies_4u.append([m_id, round(score, 4)])
    
    return top_movies_4u

In [69]:
my_movie = (get_recommendations(33, df_users_3))
my_movie

[['Inception', 0.6936],
 ['Who Framed Roger Rabbit?', 0.6634],
 ['Tangled', 0.6626],
 ['Robots', 0.6615],
 ['Mulan', 0.6582],
 ['Interstate 60', 0.657],
 ['Osmosis Jones', 0.6529],
 ['Lion King, The', 0.6477],
 ['Enchanted', 0.6455],
 ['Harry Potter and the Deathly Hallows: Part 2', 0.6447]]

In [ ]:
my_movie = (get_recommendations(3, df_users_3))
my_movie

[['Mulan ', 0.7808],
 ['Enchanted ', 0.7783],
 ['Who Framed Roger Rabbit? ', 0.7569],
 ['The Lego Movie ', 0.7366],
 ['Ice Age: Dawn of the Dinosaurs ', 0.7355],
 ['Inside Out ', 0.7335],
 ['Shrek 2 ', 0.7307],
 ['Shrek ', 0.7298],
 ['All Dogs Go to Heaven 2 ', 0.7254],
 ['Tangled ', 0.7247]]